## Monte Carlo Results

In [52]:
setwd("/home/leoKraushaar/Documents/School/Year 3/Semester 2/STAT 413/Project/protests/")
library(dplyr)

In [53]:
data <- read.csv("data/montecarlo/2030.csv")[, -1]
head(data)

,prov,season,retail,protests
,<chr>,<chr>,<dbl>,<int>
1,Alberta,Spring,10574094,9
2,British Columbia,Spring,11371951,16
3,Manitoba,Spring,3225536,9
4,New Brunswick,Spring,4129786,3
5,Newfoundland and Labrador,Spring,3186605,3
6,Northwest Territories,Spring,2056920,0


In [54]:
floor(nrow(data)/143)

[1] 8391

In [55]:
nrow(data)

[1] 1200000

In [56]:
iterations <- rep(1:8392, each=143)

data$iter <- iterations[1:nrow(data)]

data[, c("season", "retail")] <- NULL

head(data)

,prov,protests,iter
,<chr>,<int>,<int>
1,Alberta,9,1
2,British Columbia,16,1
3,Manitoba,9,1
4,New Brunswick,3,1
5,Newfoundland and Labrador,3,1
6,Northwest Territories,0,1


In [57]:
interval <- function(X, alpha=0.05) {
    quantiles <- quantile(X, probs = c(alpha/2, 1/2, 1-(alpha/2)))
    return(quantiles)
}

In [58]:
interval_low <- function(X, alpha=0.05) {
    quantiles <- quantile(X, probs = c(alpha/2, 1/2, 1-(alpha/2)))
    return(as.numeric(quantiles[1]))
}

interval_high <- function(X, alpha=0.05) {
    quantiles <- quantile(X, probs = c(alpha/2, 1/2, 1-(alpha/2)))
    return(quantiles)
    return(as.numeric(quantiles[2]))
}

In [59]:
total_protests <- data %>% group_by(prov, iter) %>% summarise(total = sum(protests))

`summarise()` has grouped output by 'prov'. You can override using the
`.groups` argument.


prov,iter,total
<chr>,<int>,<int>
Alberta,1,117
Alberta,2,128
Alberta,3,116
Alberta,4,115
Alberta,5,121
Alberta,6,129


In [60]:
intervals <- aggregate(total ~ prov, data=total_protests, FUN=interval)$total
provs <- aggregate(total ~ prov, data=total_protests, FUN=interval)$prov

In [63]:
results <- as.data.frame(cbind(prov=provs, intervals))
results

prov,2.5%,50%,97.5%
<chr>,<chr>,<chr>,<chr>
Alberta,102,117,133
British Columbia,190,219,250
Manitoba,62,72,82
New Brunswick,37,43,49
Newfoundland and Labrador,25,30,35
Northwest Territories,0,1,3
Nova Scotia,42,49,55
Nunavut,2,5,7
Ontario,661,761,863
